In [ ]:
import math
import random

In [ ]:
def distL2(x1,y1, x2,y2):
    xdiff = x2 - x1
    ydiff = y2 - y1
    return int(math.sqrt(xdiff*xdiff + ydiff*ydiff) + .5)

In [ ]:
def distL1(x1,y1, x2,y2):
    return int(abs(x2-x1) + abs(y2-y1)+.5)

In [ ]:
def calc_matriz(coord, dist):
    n = len(coord)
    D = {}
    for i in range(n-1):
        for j in range(i+1,n):
            (x1,y1) = coord[i]
            (x2,y2) = coord[j]
            D[i,j] = dist(x1,y1,x2,y2)
            D[j,i] = D[i,j]
    return n,D

In [ ]:
def read_tsplib(filename):
    f = open(filename, 'r');

    line = f.readline()
    while line.find("EDGE_WEIGHT_TYPE") == -1:
        line = f.readline()

    if line.find("EUC_2D") != -1:
        dist = distL2
    elif line.find("MAN_2D") != -1:
        dist = distL1
    else:
        print("cannot deal with non-euclidean or non-manhattan distances")
        raise Exception

    while line.find("NODE_COORD_SECTION") == -1:
        line = f.readline()

    xy_positions = []
    while 1:
        line = f.readline()
        if line.find("EOF") != -1: break
        (i,x,y) = line.split()

        x = float(x)
        y = float(y)
        xy_positions.append((x,y))

    n,D = calc_matriz(xy_positions, dist)
    return n, xy_positions, D

In [ ]:
def custo(tour):
    z = D[tour[-1], tour[0]]
    for i in range(1,len(tour)):
        z += D[tour[i], tour[i-1]]
    return z,

In [ ]:
def randtour(n):
    sol = list(range(n))
    random.shuffle(sol)
    return sol


In [ ]:
dataset = 'a280.tsp'
path = './'+dataset
print(path)

n, coord, D = read_tsplib(path)

./a280.tsp


In [ ]:
print("Solução aletória:")
tour = randtour(n)
z = custo(tour)
print(f'Rota: {tour}\ncusto: {z}')

Solução aletória:
Rota: [106, 5, 45, 248, 166, 244, 262, 150, 61, 139, 232, 142, 206, 227, 88, 123, 170, 119, 207, 231, 217, 279, 184, 219, 189, 229, 93, 254, 222, 245, 77, 152, 233, 113, 209, 28, 247, 83, 176, 253, 171, 18, 97, 149, 132, 23, 72, 54, 21, 272, 198, 63, 60, 175, 196, 79, 42, 178, 66, 199, 122, 29, 47, 24, 12, 135, 101, 223, 162, 100, 266, 99, 56, 273, 154, 125, 251, 157, 140, 270, 249, 201, 34, 103, 144, 226, 145, 195, 59, 57, 237, 274, 84, 6, 138, 8, 173, 141, 4, 261, 53, 25, 37, 200, 89, 260, 174, 268, 186, 211, 43, 167, 22, 194, 69, 193, 136, 0, 9, 257, 177, 85, 218, 46, 252, 224, 256, 163, 267, 234, 14, 92, 68, 236, 32, 121, 169, 38, 115, 158, 114, 48, 127, 55, 250, 161, 221, 275, 51, 105, 181, 27, 15, 81, 155, 187, 133, 265, 11, 50, 277, 212, 41, 215, 210, 35, 78, 216, 241, 239, 102, 124, 204, 269, 67, 87, 75, 238, 19, 116, 16, 271, 91, 65, 20, 3, 80, 129, 147, 71, 40, 159, 107, 118, 96, 62, 7, 259, 128, 70, 214, 74, 146, 131, 76, 111, 49, 213, 185, 36, 1, 151, 30, 

In [ ]:
!pip install deap

In [ ]:
import random
import numpy
from deap import base
from deap import creator
from deap import algorithms
from deap import tools

In [ ]:
toolbox = base.Toolbox()

creator.create("Minimizacao", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.Minimizacao)

toolbox.register("aleatorios", random.sample, range(n), n)
toolbox.register("gene", tools.initIterate, creator.Individual, toolbox.aleatorios)
toolbox.register("population", tools.initRepeat, list, toolbox.gene)

toolbox.register("evaluate", custo)
toolbox.register("mate", tools.cxUniformPartialyMatched, indpb=0.07)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)


In [ ]:
def statistics(individual):
  return individual.fitness.values

In [ ]:
vezes = 10

for n in range(vezes):

    populacao = toolbox.population(100)

    prob_cross = 0.8
    prob_mut = 0.2
    geracoes = 100


    stats = tools.Statistics(statistics)
    stats.register("média", numpy.mean)
    stats.register("desvio padrão", numpy.std)

    populacao, info = algorithms.eaSimple(
        populacao,
        toolbox,
        prob_cross,
        prob_mut,
        geracoes,
        stats,
        verbose=False
)

    melhores = tools.selBest(populacao, 1)

    for individuo in melhores:
        print(individuo)
        print("Melhor Solução:", individuo.fitness)
        print("--------------------------------------")
        estatisticas = stats.compile(populacao)
        print(estatisticas)

[159, 124, 19, 127, 6, 154, 132, 153, 227, 137, 82, 67, 85, 88, 108, 93, 84, 117, 234, 212, 145, 123, 130, 134, 131, 135, 209, 105, 26, 80, 111, 76, 126, 262, 270, 199, 275, 136, 22, 27, 23, 3, 128, 238, 1, 194, 223, 242, 244, 33, 39, 149, 59, 114, 197, 195, 219, 226, 261, 236, 211, 230, 260, 144, 168, 92, 87, 191, 267, 225, 213, 246, 18, 272, 259, 138, 268, 89, 100, 96, 60, 61, 30, 31, 35, 45, 28, 48, 73, 17, 152, 118, 75, 44, 122, 206, 210, 207, 174, 86, 107, 34, 36, 55, 69, 231, 83, 71, 278, 217, 245, 229, 142, 120, 257, 241, 2, 250, 279, 4, 185, 169, 187, 178, 109, 64, 79, 158, 146, 252, 204, 0, 256, 183, 103, 62, 66, 175, 104, 196, 172, 112, 170, 188, 99, 184, 141, 166, 164, 101, 41, 49, 95, 161, 186, 162, 171, 218, 163, 97, 106, 232, 277, 215, 239, 253, 248, 273, 180, 160, 182, 8, 251, 237, 208, 176, 133, 240, 43, 56, 113, 222, 269, 14, 263, 21, 32, 65, 38, 52, 24, 140, 157, 11, 16, 42, 77, 151, 110, 102, 173, 50, 68, 53, 90, 224, 214, 271, 129, 29, 189, 115, 116, 37, 7, 276, 5, 